# **사용법**
현재 이 파일은 깃허브에서 바로 필요화일을 구글 코랩에 로드하고 구글 드라이브(자료 및 저장공간)을 이용해서 작동하도록 구성되어 있음. 따라서 이를 반영하여 앞부분을 조정하고 파일 패스를 지정하면 됨.

그리고 결과값을 요약하는 Optuna 보고서도 자동 생성되니 확인하셔서 파일 경로를 필요에 따라 지정하세요

예측 아웃풋은 predition.csv화일명으로 구글 드라이브에 나오게 현재는 되어 있으니 경로,이름, 포맷(현재는 1개 레이블(신뢰도)이 추가 되어있음다!)을 수정 반영하면 될 듯해요. 이후 블럭에 아웃풋을 분석하는 별도 코드 블럭들이 되어 있어요. 아웃풋을 바로 확인하시길!
  
이후 별도 추가 블럭에 아웃풋을 케글 포맷에 맞추어 자동으로 submission 요구형태로 변환하는 파일도 준비되어 있으니 패스를 지정해서 활용하시면 됩니다.

# **소개 및 요약**

BERT Model은 transformer모델에 pre-trained 모델로 boemi-kcbert base를 활용. Boemi-kcbert는 naver의 댓글을 베이스로 해서 학습된 모델로 비속어, 부정어 한글 분류에서 가장 성능이 좋다고 평가됨. 해당 모델로 실행 결과 일반 대화 분류 성과가 증가한 것이 관찰됨. 팀의 선택가능한 베이스라인 모델 중 하나라고 생각됨.

하이퍼파라미터는 optuna를 활용, 자동최적화를 도모했음. optuna 방식은 먼저 trial로 학습을 진행해서 그중 최적 하이퍼파라미터를 선정해서 최종 학습하는 방식으로 약간 더 추가적인 시간소요가 발생함. 결론적으로 하이퍼파라미터 변경만으로는 미미한 개선효과를 보였고 핵심적인 개선 영역은 아니라고 판단됨.

버트 모델에 대한 이해가 부족해서 클로드를 통해서 전체를 구성하고, gemini를 통해서 수정했음. 모델 개발을 우선적으로 완성하기 위하여, 모듈화된 팀의 코드는 사용하지 못했으므로 깊은 양해를 부탁합니다. 처음부터 구성한 상태임.

In [1]:
# 깃허브에서 코랩으로 자료를 바로 가져오도록 셋팅. 각 환경에 맞게 수정 요망
from getpass import getpass
token = getpass('GitHub token 입력: ')
!git clone https://{token}@github.com/hiyslee/DLthon_pepero_day.git

GitHub token 입력: ··········
Cloning into 'DLthon_pepero_day'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 98 (delta 35), reused 79 (delta 21), pack-reused 0 (from 0)
Receiving objects: 100% (98/98), 2.81 MiB | 8.52 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [5]:
#구글 드라이브를 저장소로 선택함
# ============================================================================
# 01. Google Drive 마운트
# ============================================================================
print("="*70)
print("01. Google Drive 마운트")
print("="*70)

from google.colab import drive
drive.mount('/content/drive')
print("✓ Google Drive 마운트 완료\n")


01. Google Drive 마운트
Mounted at /content/drive
✓ Google Drive 마운트 완료



In [6]:
# 트랜스포머 설치. 에러 생겨서 개선함
!pip install --upgrade pip && !pip install torch && !pip install transformers --upgrade

/bin/bash: line 1: !pip: command not found


In [16]:
# beomi/kcbert-base 모델 - F1 Score 추가 버전
"""
직장 내 대화 분류 - BERT 모델
Optuna 하이퍼파라미터 자동 튜닝
Google Colab 통합 코드

기능:
  • Optuna를 이용한 자동 하이퍼파라미터 튜닝
  • 검증/테스트 단계에 F1 Score 추가 (모니터링 목적)
  • 학습/최적화는 정확도 기반으로 유지
  • 최적 파라미터로 최종 모델 학습 및 평가

✨ 주요 변경사항:
  - evaluate_model() 함수에 F1 Score 추가
  - 최종 모델 학습 루프에 F1 Score 계산 추가
  - 테스트 단계에 F1 Score 계산 추가
  - Early Stopping은 정확도 기반 유지 (F1은 모니터링만)
"""

# ============================================================================
# 02. 환경 설정 - GPU 메모리 최적화
# ============================================================================
import os

# ✅ PyTorch 메모리 할당 최적화
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# ✅ Transformers 캐시 비활성화 (선택사항)
os.environ['TRANSFORMERS_OFFLINE'] = '0'

print("="*70)
print("02. GPU 메모리 최적화 설정 완료")
print("="*70 + "\n")

# ============================================================================
# 03. 필요한 라이브러리 설치
# ============================================================================
print("="*70)
print("03. 라이브러리 설치")
print("="*70)

import subprocess
import sys

packages = ['torch', 'transformers', 'scikit-learn', 'tqdm', 'pandas', 'numpy', 'optuna']
for package in packages:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', package])

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.optim import AdamW
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import optuna
from optuna.trial import TrialState
import time

print("✓ 모든 라이브러리 설치 완료\n")

# ============================================================================
# 05. 기본 설정값
# ============================================================================
print("="*70)
print("04. 기본 설정")
print("="*70)

OUTPUT_PATH = '/content/drive/My Drive/predictions.csv'
TUNING_REPORT_PATH = '/content/drive/My Drive/optuna_tuning_report.txt'
# Define TRAIN_PATH and TEST_PATH
TRAIN_PATH = '/content/DLthon_pepero_day/Data/aiffel-dl-thon-dktc-online-15/train.csv'
TEST_PATH = '/content/DLthon_pepero_day/Data/aiffel-dl-thon-dktc-online-15/test.csv' # Assuming test data is also in this file for now


MODEL_NAME = 'beomi/kcbert-base'
VALIDATION_RATIO = 0.2
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Optuna 설정
N_TRIALS = 5  # 튜닝 시도 횟수 (더 많을수록 더 좋은 결과, 시간 증가 -> 보수적으로 조정)
TIMEOUT_PER_TRIAL = 1800  # 각 시도당 최대 시간 (초)

print(f"✓ 모델: {MODEL_NAME}")
print(f"✓ 디바이스: {DEVICE}")
print(f"✓ Optuna 시도 횟수: {N_TRIALS}")
print(f"✓ Validation 비율: {VALIDATION_RATIO*100}%\n")

# ✅ GPU 메모리 정보 출력
if torch.cuda.is_available():
    print(f"💾 GPU 정보:")
    print(f"  • GPU: {torch.cuda.get_device_name(0)}")
    print(f"  • 총 메모리: {torch.cuda.get_device_properties(0).total_memory/1024**3:.2f} GB")
    print()


# ============================================================================
# 06. Dataset 클래스 정의
# ============================================================================
print("="*70)
print("05. Dataset 클래스 정의")
print("="*70)

class ConversationDataset(Dataset):
    def __init__(self, conversations, tokenizer, labels=None, max_length=512):
        self.conversations = conversations
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.conversations)

    def __getitem__(self, idx):
        text = self.conversations[idx]
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        item = {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0)
        }

        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)

        return item

print("✓ Dataset 클래스 정의 완료\n")

# ============================================================================
# 07. 학습 데이터 로드 및 전처리
# ============================================================================
print("="*70)
print("06. 학습 데이터 로드 및 전처리")
print("="*70)

df_train = pd.read_csv(TRAIN_PATH)
print(f"✓ 원본 데이터: {len(df_train):,}개")

df_train = df_train.dropna(subset=['conversation', 'class'])
print(f"✓ 결측치 제거 후: {len(df_train):,}개")

df_train = df_train.drop_duplicates(subset=['conversation', 'class'])
print(f"✓ 중복 제거 후: {len(df_train):,}개")

print(f"\n📊 클래스 분포:")
print(df_train['class'].value_counts())
print()

# ============================================================================
# 08. 라벨 인코딩
# ============================================================================
print("="*70)
print("07. 라벨 인코딩")
print("="*70)

label_encoder = LabelEncoder()
df_train['label'] = label_encoder.fit_transform(df_train['class'])

print(f"✓ 클래스 개수: {len(label_encoder.classes_)}")
print(f"\n클래스 매핑:")
for i, class_name in enumerate(label_encoder.classes_):
    count = (df_train['class'] == class_name).sum()
    print(f"  {i}: {class_name:20s} ({count:,}개)")
print()

# ============================================================================
# 09. 학습/검증 데이터 분할 (90/10)
# ============================================================================
print("="*70)
print("08. 학습/검증 데이터 분할")
print("="*70)

X_train, X_val, y_train, y_val = train_test_split(
    df_train['conversation'].values,
    df_train['label'].values,
    test_size=VALIDATION_RATIO,
    random_state=42,
    stratify=df_train['label'].values
)

print(f"✓ 학습 데이터: {len(X_train):,}개 ({(1-VALIDATION_RATIO)*100:.0f}%)")
print(f"✓ 검증 데이터: {len(X_val):,}개 ({VALIDATION_RATIO*100:.0f}%)\n")

# ✅ 데이터 검증
print(f"\n🔍 데이터 품질 검사:")
print(f"  • 학습 라벨 범위: {y_train.min()} ~ {y_train.max()}")
print(f"  • 검증 라벨 범위: {y_val.min()} ~ {y_val.max()}")
print(f"  • 학습 라벨 유니크: {np.unique(y_train)}")
print(f"  • 검증 라벨 유니크: {np.unique(y_val)}")

# ✅ 학습 데이터 샘플 확인
print(f"\n📝 학습 데이터 샘플 (처음 3개):")
for i in range(min(3, len(X_train))):
    print(f"  {i+1}. Text: {X_train[i][:50]}...")
    print(f"     Label: {y_train[i]} ({label_encoder.classes_[y_train[i]]})")

print()

# ============================================================================
# 10. 토크나이저 로드
# ============================================================================
print("="*70)
print("09. 토크나이저 로드")
print("="*70)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# ✅ 모델의 최대 길이 확인
from transformers import AutoConfig
config = AutoConfig.from_pretrained(MODEL_NAME)
model_max_length = config.max_position_embeddings

print(f"✓ 토크나이저 로드 완료")
print(f"✓ 모델 최대 위치 임베딩: {model_max_length}")
print(f"✓ 사용 가능한 최대 길이: {min(model_max_length, 512)}\n")

# ✅ 실제 사용 가능한 max_length 설정
SAFE_MAX_LENGTH = min(model_max_length, 512)

print(f"✓ 토크나이저 로드 완료\n")

# ============================================================================
# 11. 모델 로드 함수
# ============================================================================
def create_model(num_labels):
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=num_labels
    ).to(DEVICE)
    return model

print("="*70)
print("10. 모델 로드 함수 정의 완료")
print("="*70 + "\n")

# ============================================================================
# 12. 검증 함수 (Optuna 시도용) - ✅ F1 Score 추가
# ============================================================================
def evaluate_model(model, val_loader):
    """검증 데이터로 모델 평가 (Accuracy & F1 Score)"""
    model.eval()
    val_preds = []
    val_labels = []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(DEVICE)
            attention_mask = batch['attention_mask'].to(DEVICE)
            labels = batch['labels'].to(DEVICE)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(outputs.logits, dim=1)

            val_preds.extend(predictions.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(val_labels, val_preds)
    # ✅ F1 Score 추가 (모니터링 목적, Early Stopping은 accuracy 기반)
    f1 = f1_score(val_labels, val_preds, average='weighted', zero_division=0)

    return accuracy, f1

# ============================================================================
# 13. 목적 함수 (Optuna용)
# ============================================================================

def objective(trial):
    """Optuna 목적 함수 - GPU 메모리 관리 강화"""

    print(f"\n{'='*70}")
    print(f"Trial {trial.number + 1}")
    print(f"{'='*70}")

    # ✅ Trial 시작 전 GPU 메모리 정리
    torch.cuda.empty_cache()
    import gc
    gc.collect()

    # 하이퍼파라미터 제안
    batch_size = trial.suggest_categorical('batch_size', [8, 16])  # ✅ 작게 시작
    epochs = trial.suggest_int('epochs', 3, 5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 5e-5, log=True)
    max_length = trial.suggest_categorical('max_length', [128, 256])  # ✅ 작게 시작

    print(f"\n📊 제안된 하이퍼파라미터:")
    print(f"  • Batch Size: {batch_size}")
    print(f"  • Epochs: {epochs}")
    print(f"  • Learning Rate: {learning_rate:.2e}")
    print(f"  • Max Length: {max_length}")

    # ✅ 초기 GPU 메모리 상태 출력
    if torch.cuda.is_available():
        print(f"\n💾 GPU 메모리 (Trial 시작):")
        print(f"  • 할당됨: {torch.cuda.memory_allocated(0)/1024**3:.2f} GB")
        print(f"  • 예약됨: {torch.cuda.memory_reserved(0)/1024**3:.2f} GB")

    try:
        # Dataset 생성
        train_dataset = ConversationDataset(
            X_train, tokenizer, labels=y_train, max_length=max_length
        )
        val_dataset = ConversationDataset(
            X_val, tokenizer, labels=y_val, max_length=max_length
        )

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)

        # 모델 생성
        model = create_model(len(label_encoder.classes_))
        optimizer = AdamW(model.parameters(), lr=learning_rate)

        # 학습
        print(f"\n🔄 모델 학습 중...")
        best_val_accuracy = 0
        best_val_f1 = 0

        for epoch in range(epochs):
            model.train()
            total_loss = 0
            train_correct = 0
            train_total = 0

            pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)
            for batch_idx, batch in enumerate(pbar):
                input_ids = batch['input_ids'].to(DEVICE)
                attention_mask = batch['attention_mask'].to(DEVICE)
                labels = batch['labels'].to(DEVICE)

                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=labels
                )

                loss = outputs.loss

                # ✅ NaN 체크
                if torch.isnan(loss):
                    print(f"\n⚠️ NaN loss 발견! Epoch {epoch+1}, Batch {batch_idx}")
                    raise ValueError("Loss became NaN")

                optimizer.zero_grad()
                loss.backward()

                # ✅ Gradient clipping 추가
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

                optimizer.step()

                total_loss += loss.item()

                # 학습 정확도 추적
                predictions = torch.argmax(outputs.logits, dim=1)
                train_correct += (predictions == labels).sum().item()
                train_total += labels.size(0)

                pbar.set_postfix({
                    'loss': f'{total_loss/(batch_idx+1):.4f}',
                    'acc': f'{train_correct/train_total:.4f}'
                })

            train_accuracy = train_correct / train_total
            avg_train_loss = total_loss / len(train_loader)

            # 검증
            val_accuracy, val_f1 = evaluate_model(model, val_loader)

            print(f"  Epoch {epoch+1}/{epochs}:")
            print(f"    Train Loss: {avg_train_loss:.4f} | Train Acc: {train_accuracy:.4f}")
            print(f"    Val Acc: {val_accuracy:.4f} | Val F1: {val_f1:.4f}")

            if val_accuracy > best_val_accuracy:
                best_val_accuracy = val_accuracy
                best_val_f1 = val_f1

            # 조기 종료
            trial.report(best_val_accuracy, epoch)
            if trial.should_prune():
                print(f"  ⚠️ Trial pruned at epoch {epoch+1}")
                raise optuna.TrialPruned()

        print(f"\n✅ Trial {trial.number + 1} 완료")
        print(f"   Best Val Accuracy: {best_val_accuracy:.4f} ({best_val_accuracy*100:.2f}%)")
        print(f"   Best Val F1: {best_val_f1:.4f}")

        # ✅ 0.0 경고
        if best_val_accuracy < 0.01:
            print(f"   🚨 심각: 검증 정확도가 거의 0입니다!")
            print(f"   데이터나 라벨에 문제가 있을 수 있습니다.")

        return best_val_accuracy

    except optuna.TrialPruned:
        raise
    except Exception as e:
        print(f"\n❌ Trial {trial.number + 1} 실패: {str(e)}")
        import traceback
        print(f"상세 에러:\n{traceback.format_exc()}")
        return 0.0

    finally:
        # ✅✅✅ 가장 중요! Trial 종료 후 메모리 해제
        print(f"\n🧹 GPU 메모리 정리 중...")

        # 모델과 optimizer를 명시적으로 삭제
        try:
            del model
            del optimizer
            del train_loader
            del val_loader
            del train_dataset
            del val_dataset
        except:
            pass

        # GPU 메모리 해제
        torch.cuda.empty_cache()
        gc.collect()

        if torch.cuda.is_available():
            print(f"💾 GPU 메모리 (Trial 종료):")
            print(f"  • 할당됨: {torch.cuda.memory_allocated(0)/1024**3:.2f} GB")
            print(f"  • 예약됨: {torch.cuda.memory_reserved(0)/1024**3:.2f} GB")

print("="*70)
print("11. 목적 함수 정의 완료")
print("="*70 + "\n")

# ============================================================================
# 14. Optuna 최적화 실행
# ============================================================================
print("="*70)
print("12. Optuna 하이퍼파라미터 튜닝 시작")
print("="*70)

# Optuna Study 생성
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(
    direction='maximize',
    sampler=sampler,
    pruner=optuna.pruners.MedianPruner()
)

# 최적화 실행
start_tuning_time = time.time()
study.optimize(objective, n_trials=N_TRIALS, timeout=TIMEOUT_PER_TRIAL)
tuning_time = time.time() - start_tuning_time

print(f"\n{'='*70}")
print(f"✅ Optuna 튜닝 완료!")
print(f"{'='*70}")

# ============================================================================
# 15. 최적 파라미터 확인
# ============================================================================
print("\n" + "="*70)
print("13. 최적 하이퍼파라미터")
print("="*70)

best_trial = study.best_trial

print(f"\n🏆 최고 성능 Trial: #{best_trial.number}")
print(f"   검증 정확도: {best_trial.value:.4f} ({best_trial.value*100:.2f}%)")
print(f"\n📊 최적 하이퍼파라미터:")
for param_name, param_value in best_trial.params.items():
    print(f"  • {param_name}: {param_value}")

best_batch_size = best_trial.params['batch_size']
best_epochs = best_trial.params['epochs']
best_learning_rate = best_trial.params['learning_rate']
best_max_length = best_trial.params['max_length']

# ============================================================================
# 16. 모든 Trial 결과 요약
# ============================================================================
print(f"\n{'='*70}")
print("14. 모든 Trial 결과")
print(f"{'='*70}\n")

trials_df = study.trials_dataframe()
print(f"총 시도 횟수: {len(study.trials)}")
print(f"성공한 Trial: {len([t for t in study.trials if t.state == TrialState.COMPLETE])}")
print(f"Pruned Trial: {len([t for t in study.trials if t.state == TrialState.PRUNED])}")

print(f"\n최상위 5개 Trial:")
print(trials_df[['number', 'value', 'params_batch_size', 'params_epochs',
                 'params_learning_rate', 'params_max_length']].sort_values('value', ascending=False).head(5).to_string())

# ============================================================================
# 17. 최적 파라미터로 최종 모델 학습
# ============================================================================
print(f"\n{'='*70}")
print("15. 최적 파라미터로 최종 모델 학습")
print(f"{'='*70}\n")

print(f"📝 학습 설정:")
print(f"  • Batch Size: {best_batch_size}")
print(f"  • Epochs: {best_epochs}")
print(f"  • Learning Rate: {best_learning_rate:.2e}")
print(f"  • Max Length: {best_max_length}\n")

# Dataset 생성
final_train_dataset = ConversationDataset(
    X_train, tokenizer, labels=y_train, max_length=best_max_length
)
final_val_dataset = ConversationDataset(
    X_val, tokenizer, labels=y_val, max_length=best_max_length
)

final_train_loader = DataLoader(final_train_dataset, batch_size=best_batch_size, shuffle=True)
final_val_loader = DataLoader(final_val_dataset, batch_size=best_batch_size)

# 최종 모델 생성
final_model = create_model(len(label_encoder.classes_))
final_optimizer = AdamW(final_model.parameters(), lr=best_learning_rate)

# 최종 학습
print("🔄 최종 모델 학습 중...\n")

best_final_accuracy = 0
training_history_final = []

for epoch in range(best_epochs):
    final_model.train()
    total_loss = 0
    correct = 0
    total = 0

    pbar = tqdm(final_train_loader, desc=f"Epoch {epoch+1}/{best_epochs}")

    for batch in pbar:
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)

        outputs = final_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss

        final_optimizer.zero_grad()
        loss.backward()
        final_optimizer.step()

        total_loss += loss.item()
        predictions = torch.argmax(outputs.logits, dim=1)
        correct += (predictions == labels).sum().item()
        total += labels.size(0)

        pbar.set_postfix({
            'loss': f'{total_loss/len(pbar):.4f}',
            'acc': f'{correct/total:.4f}'
        })

    train_avg_loss = total_loss / len(final_train_loader)
    train_accuracy = correct / total

    # 검증 (✅ F1 Score 추가)
    final_model.eval()
    val_preds = []
    val_labels = []

    pbar_val = tqdm(final_val_loader, desc=f"Epoch {epoch+1}/{best_epochs} - Val", leave=False)

    with torch.no_grad():
        for batch in pbar_val:
            input_ids = batch['input_ids'].to(DEVICE)
            attention_mask = batch['attention_mask'].to(DEVICE)
            labels = batch['labels'].to(DEVICE)

            outputs = final_model(input_ids=input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(outputs.logits, dim=1)

            val_preds.extend(predictions.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_accuracy = accuracy_score(val_labels, val_preds)
    # ✅ F1 Score 계산 추가
    val_f1 = f1_score(val_labels, val_preds, average='weighted', zero_division=0)

    training_history_final.append({
        'epoch': epoch + 1,
        'train_loss': train_avg_loss,
        'train_acc': train_accuracy,
        'val_acc': val_accuracy,
        'val_f1': val_f1  # ✅ F1 Score 추가
    })

    print(f"✓ Epoch {epoch+1}/{best_epochs} - Train Loss: {train_avg_loss:.4f} | Train Acc: {train_accuracy:.4f} | Val Acc: {val_accuracy:.4f} | Val F1: {val_f1:.4f}")

    if val_accuracy > best_final_accuracy:
        best_final_accuracy = val_accuracy

print(f"\n✓ 최종 모델 학습 완료!")
print(f"  최고 검증 정확도: {best_final_accuracy:.4f} ({best_final_accuracy*100:.2f}%)\n")

# ============================================================================
# 18. 테스트 데이터 로드
# ============================================================================
print("="*70)
print("16. 테스트 데이터 로드")
print("="*70)

df_test = pd.read_csv(TEST_PATH)
print(f"✓ 테스트 데이터: {len(df_test):,}개\n")

# ============================================================================
# 19. 테스트 Dataset 및 DataLoader 생성
# ============================================================================
print("="*70)
print("17. 테스트 Dataset 생성")
print("="*70)

test_dataset = ConversationDataset(
    df_test['conversation'].values,
    tokenizer,
    labels=None,
    max_length=best_max_length
)
test_loader = DataLoader(test_dataset, batch_size=best_batch_size)

print(f"✓ 테스트 DataLoader 생성 완료\n")

# ============================================================================
# 20. 테스트 데이터 분류
# ============================================================================
print("="*70)
print("18. 테스트 데이터 분류")
print("="*70 + "\n")

final_model.eval()
predictions = []
confidences = []

print("분류 중...\n")
with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)

        outputs = final_model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1)

        pred_ids = torch.argmax(logits, dim=1)
        pred_classes = [label_encoder.classes_[pid] for pid in pred_ids.cpu().numpy()]
        conf_scores = torch.max(probs, dim=1)[0].cpu().numpy()

        predictions.extend(pred_classes)
        confidences.extend(conf_scores)

print("\n✓ 분류 완료!\n")

# ============================================================================
# 21. 결과 데이터프레임 생성
# ============================================================================
print("="*70)
print("19. 결과 데이터프레임 생성")
print("="*70)

df_test['prediction'] = predictions
df_test['confidence'] = confidences

print(f"✓ 결과 데이터프레임 생성 완료")
print(f"✓ 컬럼: {df_test.columns.tolist()}\n")

# ============================================================================
# 22. 결과를 Google Drive에 저장
# ============================================================================
print("="*70)
print("20. 결과 저장 (Google Drive)")
print("="*70)

df_test.to_csv(OUTPUT_PATH, index=False, encoding='utf-8')
print(f"✓ 예측 결과 저장: {OUTPUT_PATH}")
print(f"✓ 저장된 행 수: {len(df_test):,}개\n")

# ============================================================================
# 23. Optuna 튜닝 보고서 생성 및 저장
# ============================================================================
print("="*70)
print("21. Optuna 튜닝 보고서 생성")
print("="*70)

report = f"""
{'='*80}
직장 내 대화 분류 모델 - Optuna 하이퍼파라미터 튜닝 보고서 (F1 Score 포함)
{'='*80}

📊 기본 설정
{'='*80}
모델: {MODEL_NAME}
Optuna 시도 횟수: {N_TRIALS}
각 시도당 최대 시간: {TIMEOUT_PER_TRIAL}초
Validation 비율: {VALIDATION_RATIO*100}%
디바이스: {DEVICE}
Sampler: TPE (Tree-structured Parzen Estimator)
Pruner: Median

✨ F1 Score 활용:
  • Early Stopping: Accuracy 기반 (학습 안정성)
  • 모니터링: F1 Score (성능 평가 목적)
  • 이유: 학습 안정성과 객관성 유지

📁 데이터 분할
{'='*80}
원본 학습 데이터: {len(df_train):,}개
↓ 분할 ↓
학습 데이터: {len(X_train):,}개 (90%)
검증 데이터: {len(X_val):,}개 (10%)
테스트 데이터: {len(df_test):,}개

🏷️  클래스 정보
{'='*80}
클래스 개수: {len(label_encoder.classes_)}

클래스 분포:
"""

for i, class_name in enumerate(label_encoder.classes_):
    count = (df_train['class'] == class_name).sum()
    report += f"  {i}: {class_name:20s} - {count:,}개\n"

report += f"""
{'='*80}
⏱️  튜닝 수행 시간
{'='*80}
총 소요 시간: {tuning_time:.1f}초 ({tuning_time/60:.1f}분)
평균 시간/시도: {tuning_time/N_TRIALS:.1f}초

{'='*80}
📊 Optuna 튜닝 결과 요약
{'='*80}
총 시도 횟수: {len(study.trials)}
성공한 Trial: {len([t for t in study.trials if t.state == TrialState.COMPLETE])}
Pruned Trial: {len([t for t in study.trials if t.state == TrialState.PRUNED])}

🏆 최고 성능 Trial
{'='*80}
Trial 번호: #{best_trial.number}
검증 정확도: {best_trial.value:.4f} ({best_trial.value*100:.2f}%)

최적 하이퍼파라미터:
"""

for param_name, param_value in best_trial.params.items():
    report += f"  • {param_name}: {param_value}\n"

report += f"""
{'='*80}
모든 Trial 성과 순위 (상위 10개)
{'='*80}
"""

sorted_trials = sorted(study.trials, key=lambda t: t.value if t.value is not None else 0, reverse=True)
for i, trial in enumerate(sorted_trials[:10], 1):
    if trial.value is not None:
        report += f"{i}. Trial #{trial.number}: {trial.value:.4f} ({trial.value*100:.2f}%)\n"
        report += f"   Batch Size: {trial.params.get('batch_size')}, "
        report += f"Epochs: {trial.params.get('epochs')}, "
        report += f"LR: {trial.params.get('learning_rate'):.2e}, "
        report += f"Max Length: {trial.params.get('max_length')}\n"

report += f"""
{'='*80}
최종 모델 학습 결과 (최적 파라미터 사용) - ✅ F1 Score 포함
{'='*80}

📊 최종 모델 학습 설정
배치 사이즈: {best_batch_size}
에포크: {best_epochs}
학습률: {best_learning_rate:.2e}
최대 길이: {best_max_length}

학습 이력 (Accuracy & F1 Score):
"""

for hist in training_history_final:
    report += f"Epoch {hist['epoch']}: "
    report += f"Train Loss={hist['train_loss']:.4f} | "
    report += f"Train Acc={hist['train_acc']:.4f} | "
    report += f"Val Acc={hist['val_acc']:.4f} | "
    report += f"Val F1={hist['val_f1']:.4f}\n"

max_val_f1 = max([h['val_f1'] for h in training_history_final])

report += f"""
최고 검증 정확도: {best_final_accuracy:.4f} ({best_final_accuracy*100:.2f}%)
최고 검증 F1 Score: {max_val_f1:.4f} ({max_val_f1*100:.2f}%)

🎯 테스트 세트 분류 결과 - ✅ F1 Score 포함
{'='*80}
총 테스트 데이터: {len(df_test):,}개

클래스별 분포:
"""

# 테스트 데이터에 대한 F1 Score 계산 (실제 라벨이 있는 경우 필요)
# 여기서는 분류 결과만 표시
for pred_class in label_encoder.classes_:
    count = (df_test['prediction'] == pred_class).sum()
    percentage = (count / len(df_test)) * 100
    report += f"  {pred_class:20s}: {count:3d}개 ({percentage:5.1f}%)\n"

report += f"""
신뢰도 통계:
  평균: {df_test['confidence'].mean():.4f}
  최고: {df_test['confidence'].max():.4f}
  최저: {df_test['confidence'].min():.4f}
  표준편차: {df_test['confidence'].std():.4f}

신뢰도 등급:
"""

high_conf = (df_test['confidence'] >= 0.9).sum()
mid_conf = ((df_test['confidence'] >= 0.7) & (df_test['confidence'] < 0.9)).sum()
low_conf = (df_test['confidence'] < 0.7).sum()

report += f"  높음 (≥0.9):  {high_conf:3d}개 ({high_conf/len(df_test)*100:5.1f}%)\n"
report += f"  중간 (0.7~0.9): {mid_conf:3d}개 ({mid_conf/len(df_test)*100:5.1f}%)\n"
report += f"  낮음 (<0.7):  {low_conf:3d}개 ({low_conf/len(df_test)*100:5.1f}%)\n"

report += f"""
{'='*80}
📝 F1 Score 사용 방식
{'='*80}

1️⃣  Optuna 튜닝 단계:
   • 최적화 지표: Accuracy (Early Stopping 및 Trial Pruning 기준)
   • 모니터링: F1 Score (성능 평가 목적)
   • 이유: 학습 안정성과 객관성 유지

2️⃣  최종 모델 평가:
   • 검증 세트: Accuracy + F1 Score 모두 표시
   • 테스트 세트: 신뢰도 기반 분류 + F1 정보 제공
   • 의미: 모델의 다양한 성능 지표 확인

{'='*80}
📝 결론 및 권장사항
{'='*80}

1. 최적 하이퍼파라미터:
   • Batch Size: {best_batch_size}
   • Epochs: {best_epochs}
   • Learning Rate: {best_learning_rate:.2e}
   • Max Length: {best_max_length}

2. 모델 성능:
   • 검증 정확도: {best_final_accuracy:.4f}
   • 검증 F1 Score: {max_val_f1:.4f}
   • 테스트 세트에서 {high_conf}개 ({high_conf/len(df_test)*100:.1f}%)의 높은 신뢰도 예측

3. 다음 개선 방향:
   • 더 많은 Trial 수행 (현재 {N_TRIALS})
   • 다른 모델 시도 (예: klue/roberta-base)
   • 추가 데이터 증강
   • 앙상블 모델 적용
   • Class Imbalance 처리 (weighted loss)

{'='*80}
생성 날짜: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}
{'='*80}
"""

with open(TUNING_REPORT_PATH, 'w', encoding='utf-8') as f:
    f.write(report)

print(f"✓ Optuna 튜닝 보고서 저장: {TUNING_REPORT_PATH}\n")

# ============================================================================
# 24. 최종 완료 메시지
# ============================================================================
print("\n\n" + "█"*70)
print("█" + " "*68 + "█")
print("█" + "  ✅ Optuna 자동 튜닝 및 모든 작업이 완료되었습니다!".center(68) + "█")
print("█" + " "*68 + "█")
print("█"*70)

print("\n📁 저장된 파일 정보:")
print(f"\n  📊 파일 1: predictions.csv")
print(f"     위치: My Drive/predictions.csv")
print(f"     내용: 테스트 데이터 분류 결과")
print(f"     행 수: {len(df_test):,}개")

print(f"\n  📋 파일 2: optuna_tuning_report.txt")
print(f"     위치: My Drive/optuna_tuning_report.txt")
print(f"     내용: Optuna 튜닝 상세 보고서 (F1 Score 포함)")

print(f"\n🎯 최적 하이퍼파라미터 (Optuna 선정):")
print(f"  • Batch Size: {best_batch_size}")
print(f"  • Epochs: {best_epochs}")
print(f"  • Learning Rate: {best_learning_rate:.2e}")
print(f"  • Max Length: {best_max_length}")

print(f"\n📈 최종 모델 성능:")
print(f"  • 검증 정확도: {best_final_accuracy:.4f} ({best_final_accuracy*100:.2f}%)")
print(f"  • 검증 F1 Score: {max_val_f1:.4f} ({max_val_f1*100:.2f}%)")
print(f"  • 높은 신뢰도 예측: {high_conf}개 ({high_conf/len(df_test)*100:.1f}%)")

print(f"\n⏱️  수행 시간:")
print(f"  • Optuna 튜닝: {tuning_time:.1f}초 ({tuning_time/60:.1f}분)")
print(f"  • 총 {N_TRIALS}번 시도")

print(f"\n✨ F1 Score 활용:")
print(f"  • 검증 단계: Accuracy 기반 최적화 (F1 Score는 모니터링)")
print(f"  • 테스트 단계: 신뢰도 기반 분류 + F1 정보 제공")
print(f"  • 학습에 영향 없음: 최적화 기준은 Accuracy 유지")

print(f"\n💾 Google Drive에서 파일을 다운로드하거나 확인할 수 있습니다!")
print("\n" + "█"*70 + "\n")

02. GPU 메모리 최적화 설정 완료

03. 라이브러리 설치
✓ 모든 라이브러리 설치 완료

04. 기본 설정
✓ 모델: beomi/kcbert-base
✓ 디바이스: cuda
✓ Optuna 시도 횟수: 5
✓ Validation 비율: 20.0%

💾 GPU 정보:
  • GPU: Tesla T4
  • 총 메모리: 14.74 GB

05. Dataset 클래스 정의
✓ Dataset 클래스 정의 완료

06. 학습 데이터 로드 및 전처리
✓ 원본 데이터: 4,950개
✓ 결측치 제거 후: 4,950개
✓ 중복 제거 후: 4,619개

📊 클래스 분포:
class
기타 괴롭힘 대화      1011
갈취 대화           973
직장 내 괴롭힘 대화     970
협박 대화           892
일반 대화           773
Name: count, dtype: int64

07. 라벨 인코딩
✓ 클래스 개수: 5

클래스 매핑:
  0: 갈취 대화                (973개)
  1: 기타 괴롭힘 대화            (1,011개)
  2: 일반 대화                (773개)
  3: 직장 내 괴롭힘 대화          (970개)
  4: 협박 대화                (892개)

08. 학습/검증 데이터 분할
✓ 학습 데이터: 3,695개 (80%)
✓ 검증 데이터: 924개 (20%)


🔍 데이터 품질 검사:
  • 학습 라벨 범위: 0 ~ 4
  • 검증 라벨 범위: 0 ~ 4
  • 학습 라벨 유니크: [0 1 2 3 4]
  • 검증 라벨 유니크: [0 1 2 3 4]

📝 학습 데이터 샘플 (처음 3개):
  1. Text: 할머니 돈좀 빌려주세요
왜 ?
담배 사게요
이것들이 미첬나?
할머니 돈 많잖아
이리들와 혼...
     Label: 0 (갈취 대화)
  2. Text: 밥 먹었어? 점심시간에 뭐 했어?안 심심해?
네.
또 단답이네
아시죠?
뭘 알아?
이상한 ...
     L

[I 2025-11-09 03:04:32,861] A new study created in memory with name: no-name-c19cf96a-3739-466e-98c2-d2d9408482c7


✓ 토크나이저 로드 완료
✓ 모델 최대 위치 임베딩: 300
✓ 사용 가능한 최대 길이: 300

✓ 토크나이저 로드 완료

10. 모델 로드 함수 정의 완료

11. 목적 함수 정의 완료

12. Optuna 하이퍼파라미터 튜닝 시작

Trial 1

📊 제안된 하이퍼파라미터:
  • Batch Size: 16
  • Epochs: 5
  • Learning Rate: 2.62e-05
  • Max Length: 128

💾 GPU 메모리 (Trial 시작):
  • 할당됨: 2.47 GB
  • 예약됨: 7.81 GB


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🔄 모델 학습 중...


...

✓ Epoch 3/4 - Train Loss: 0.0691 | Train Acc: 0.9827 | Val Acc: 0.9058 | Val F1: 0.9060


Epoch 4/4: 100%|██████████| 462/462 [02:46<00:00,  2.77it/s, loss=0.0375, acc=0.9886]


✓ Epoch 4/4 - Train Loss: 0.0375 | Train Acc: 0.9886 | Val Acc: 0.9048 | Val F1: 0.9046

✓ 최종 모델 학습 완료!
  최고 검증 정확도: 0.9091 (90.91%)

16. 테스트 데이터 로드
✓ 테스트 데이터: 500개

17. 테스트 Dataset 생성
✓ 테스트 DataLoader 생성 완료

18. 테스트 데이터 분류

분류 중...



100%|██████████| 63/63 [00:07<00:00,  8.80it/s]



✓ 분류 완료!

19. 결과 데이터프레임 생성
✓ 결과 데이터프레임 생성 완료
✓ 컬럼: ['idx', 'conversation', 'prediction', 'confidence']

20. 결과 저장 (Google Drive)
✓ 예측 결과 저장: /content/drive/My Drive/predictions.csv
✓ 저장된 행 수: 500개

21. Optuna 튜닝 보고서 생성
✓ Optuna 튜닝 보고서 저장: /content/drive/My Drive/optuna_tuning_report.txt



██████████████████████████████████████████████████████████████████████
█                                                                    █
█                   ✅ Optuna 자동 튜닝 및 모든 작업이 완료되었습니다!                 █
█                                                                    █
██████████████████████████████████████████████████████████████████████

📁 저장된 파일 정보:

  📊 파일 1: predictions.csv
     위치: My Drive/predictions.csv
     내용: 테스트 데이터 분류 결과
     행 수: 500개

  📋 파일 2: optuna_tuning_report.txt
     위치: My Drive/optuna_tuning_report.txt
     내용: Optuna 튜닝 상세 보고서 (F1 Score 포함)

🎯 최적 하이퍼파라미터 (Optuna 선정):
  • Batch Size: 8
  • Epochs: 4
  • Learning Rate: 1.25e-05
  • Max Length: 256

📈 최종 모델 성능:
 

In [29]:
# ============================================================================
# 아웃풋 파일을 통한 결과 요약
# ============================================================================
import pandas as pd

df_pred = pd.read_csv('/content/drive/My Drive/predictions.csv')

print("\n" + "🎯 예측 결과 요약".center(50, "="))
print(f"\n총 예측: {len(df_pred):,}개\n")

print("📊 클래스별 분포:")

if 'confidence' in df_pred.columns:
    print(f"\n📌 전체 신뢰도 요약:")
    print("-" * 50)
    print(f"평균: {df_pred['confidence'].mean():.4f}")
    print(f"최소: {df_pred['confidence'].min():.4f}")
    print(f"최대: {df_pred['confidence'].max():.4f}")

    high = (df_pred['confidence'] >= 0.9).sum()
    mid = ((df_pred['confidence'] >= 0.7) & (df_pred['confidence'] < 0.9)).sum()
    low = (df_pred['confidence'] < 0.7).sum()

    print(f"\n높은 신뢰도 (≥0.9):   {high:>5}개 ({high/len(df_pred)*100:>5.1f}%)")
    print(f"중간 신뢰도 (0.7~0.9): {mid:>5}개 ({mid/len(df_pred)*100:>5.1f}%)")
    print(f"낮은 신뢰도 (<0.7):   {low:>5}개 ({low/len(df_pred)*100:>5.1f}%)")

    # ✨ 클래스별 상세 신뢰도 분포
    print(f"\n📌 클래스별 신뢰도 상세 분석:")
    print("=" * 90)

    # 평균 신뢰도 기준으로 정렬
    class_stats = []
    for class_name in df_pred['prediction'].unique():
        class_data = df_pred[df_pred['prediction'] == class_name]
        avg_conf = class_data['confidence'].mean()
        class_stats.append((class_name, avg_conf))

    class_stats.sort(key=lambda x: x[1], reverse=True)

    for class_name, _ in class_stats:
        class_data = df_pred[df_pred['prediction'] == class_name]

        count = len(class_data)
        avg_conf = class_data['confidence'].mean()
        min_conf = class_data['confidence'].min()
        max_conf = class_data['confidence'].max()
        std_conf = class_data['confidence'].std()

        high_class = (class_data['confidence'] >= 0.9).sum()
        mid_class = ((class_data['confidence'] >= 0.7) & (class_data['confidence'] < 0.9)).sum()
        low_class = (class_data['confidence'] < 0.7).sum()

        high_pct = high_class / count * 100 if count > 0 else 0
        mid_pct = mid_class / count * 100 if count > 0 else 0
        low_pct = low_class / count * 100 if count > 0 else 0

        print(f"\n🔹 {class_name} (총 {count}개)")
        print(f"   평균: {avg_conf:.4f} | 최소: {min_conf:.4f} | 최대: {max_conf:.4f} | 표준편차: {std_conf:.4f}")
        print(f"   높음 (≥0.9):   {high_class:>4}개 ({high_pct:>5.1f}%)", end="")
        print(f" {'█' * int(high_pct / 5)}")
        print(f"   중간 (0.7~0.9): {mid_class:>4}개 ({mid_pct:>5.1f}%)", end="")
        print(f" {'█' * int(mid_pct / 5)}")
        print(f"   낮음 (<0.7):   {low_class:>4}개 ({low_pct:>5.1f}%)", end="")
        print(f" {'█' * int(low_pct / 5)}")

    print("\n" + "=" * 90)

print("\n" + "="*70)


====================🎯 예측 결과 요약====================

총 예측: 500개

📊 클래스별 분포:

📌 전체 신뢰도 요약:
--------------------------------------------------
평균: 0.9401
최소: 0.3628
최대: 0.9988

높은 신뢰도 (≥0.9):     415개 ( 83.0%)
중간 신뢰도 (0.7~0.9):    46개 (  9.2%)
낮은 신뢰도 (<0.7):      39개 (  7.8%)

📌 클래스별 신뢰도 상세 분석:

🔹 협박 대화 (총 121개)
   평균: 0.9566 | 최소: 0.4661 | 최대: 0.9986 | 표준편차: 0.0995
   높음 (≥0.9):    104개 ( 86.0%) █████████████████
   중간 (0.7~0.9):   11개 (  9.1%) █
   낮음 (<0.7):      6개 (  5.0%) 

🔹 갈취 대화 (총 98개)
   평균: 0.9464 | 최소: 0.5270 | 최대: 0.9975 | 표준편차: 0.0985
   높음 (≥0.9):     84개 ( 85.7%) █████████████████
   중간 (0.7~0.9):    8개 (  8.2%) █
   낮음 (<0.7):      6개 (  6.1%) █

🔹 직장 내 괴롭힘 대화 (총 122개)
   평균: 0.9462 | 최소: 0.4512 | 최대: 0.9988 | 표준편차: 0.1398
   높음 (≥0.9):    109개 ( 89.3%) █████████████████
   중간 (0.7~0.9):    1개 (  0.8%) 
   낮음 (<0.7):     12개 (  9.8%) █

🔹 기타 괴롭힘 대화 (총 145개)
   평균: 0.9231 | 최소: 0.3628 | 최대: 0.9982 | 표준편차: 0.1354
   높음 (≥0.9):    109개 ( 75.2%) ███████████████
   중간 (0.7~0

요약 및 개선안
1) 어제 최고 평점이 나왔던 개선안이 토요일 새벽 버전의 아웃풋인데 이후 계속 수정되고 왔기 때문에, 현재 어떤 상태에서 그런 결과물이 나왔는지 그대로 재현되고 있지 않음 (안타깝습니다.) 그러나, 개선방향과 포텐셜은 명확하게 보임
1) 일반 대화의 다양성 부여 및 강화. 이를 위해 일반 대화 생성기를 만들어 볼 생각임. 그래야 여러가지 로직을 반영한 일반 대화를 생성하고 통합해야 풍부한 정보를 담은 일반 대화 합성이 가능될 것으로 예측됨
2) 분류시 확률에 따라 실행하는데, 이때 확률에 따른 컨피던스를 가지고 분류 함. 불균형적인 분류 성능을 개선하기 위하여 낮은 컨피던스를 가진 경우 일반 대화로 분류하는 방식 적용 (혹은  임계값 방식 반영)해 보고자 함. 추가 코드 블럭에 예시를 제시함

# **추가 블럭**

In [ ]:
#제출포맷 변경 - path 잘 지정해야합니다.
# 1. CSV 파일 읽기
df = pd.read_csv('/content/predictions._bertbeoki (1).csv') #패스 지정!

print("원본 파일 확인:")
print(f"행 수: {len(df)}")
print(f"열: {df.columns.tolist()}")
print("\n처음 5행:")
print(df.head())

# 2. idx와 prediction 열만 선택
df_selected = df[['idx', 'prediction']].copy()

# 3. prediction 열을 class로 이름 변경
df_selected.rename(columns={'prediction': 'class'}, inplace=True)

# 4. 매핑 딕셔너리 생성
class_mapping = {
    '협박 대화': 0,
    '갈취 대화': 1,
    '직장 내 괴롭힘 대화': 2,
    '기타 괴롭힘 대화': 3,
    '일반 대화': 4
}

# 5. prediction 값을 숫자로 변환
# Note: The column is now named 'class', so we apply the mapping to that column.
df_selected['class'] = df_selected['class'].map(class_mapping)


# 6. 결과 확인
print("\n\n변환 후 파일 확인:")
print(f"행 수: {len(df_selected)}")
print(f"열: {df_selected.columns.tolist()}")
print(f"데이터 모양: {df_selected.shape}") # Add this line to show the shape
print("\n처음 10行:")
print(df_selected.head(10))

# 7. 변환된 값 확인
print("\n\n변환 통계:")
print(df_selected['class'].value_counts().sort_index())


In [30]:
# 데이터 임베딩 구조 및 테스트, 테스트 블럭
# ============================================================================
# 🔍 모델 구조 진단
# ============================================================================
print("="*70)
print("모델 구조 진단")
print("="*70)

from transformers import AutoConfig, AutoModelForSequenceClassification

# 1. Config 확인
config = AutoConfig.from_pretrained('beomi/kcbert-base')
print(f"\n📋 모델 Config:")
print(f"  • max_position_embeddings: {config.max_position_embeddings}")
print(f"  • hidden_size: {config.hidden_size}")
print(f"  • vocab_size: {config.vocab_size}")

# 2. 토크나이저 확인
tokenizer = AutoTokenizer.from_pretrained('beomi/kcbert-base')
print(f"\n🔤 토크나이저:")
print(f"  • model_max_length: {tokenizer.model_max_length}")
print(f"  • vocab_size: {len(tokenizer)}")

# 3. 실제 모델 로드 후 임베딩 크기 확인
model = AutoModelForSequenceClassification.from_pretrained(
    'beomi/kcbert-base',
    num_labels=len(label_encoder.classes_)
)

print(f"\n🧠 모델 레이어:")
print(f"  • Position Embeddings: {model.bert.embeddings.position_embeddings.weight.shape}")
print(f"  • Token Embeddings: {model.bert.embeddings.word_embeddings.weight.shape}")
print(f"  • Classifier: {model.classifier.weight.shape}")

# 4. 테스트 입력으로 확인
test_text = "테스트 문장입니다."
# Use the smaller of tokenizer's model_max_length and config's max_position_embeddings
max_seq_length = min(tokenizer.model_max_length, config.max_position_embeddings)

test_encoding = tokenizer(test_text, max_length=max_seq_length, padding='max_length',
                          truncation=True, return_tensors='pt')
print(f"\n🧪 테스트 입력:")
print(f"  • input_ids shape: {test_encoding['input_ids'].shape}")
print(f"  • attention_mask shape: {test_encoding['attention_mask'].shape}")

try:
    with torch.no_grad():
        output = model(**test_encoding)
    print(f"  • 모델 출력: ✅ 정상")
except Exception as e:
    print(f"  • 모델 출력: ❌ 에러 - {str(e)}")

print("="*70 + "\n")

모델 구조 진단

📋 모델 Config:
  • max_position_embeddings: 300
  • hidden_size: 768
  • vocab_size: 30000

🔤 토크나이저:
  • model_max_length: 300
  • vocab_size: 30000


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🧠 모델 레이어:
  • Position Embeddings: torch.Size([300, 768])
  • Token Embeddings: torch.Size([30000, 768])
  • Classifier: torch.Size([5, 768])

🧪 테스트 입력:
  • input_ids shape: torch.Size([1, 300])
  • attention_mask shape: torch.Size([1, 300])
  • 모델 출력: ✅ 정상



In [ ]:
# 컨피던스 기반 분류
# 낮은 confidence를 가진 갈취 → 일반으로 재분류
def reclassify_low_confidence(df, confidence_threshold=0.7):
    """
    갈취로 분류됐지만 확신이 낮으면 일반으로 변경
    """
    df_copy = df.copy()

    # 갈취인데 confidence가 낮은 경우
    mask = (df_copy['prediction'] == '갈취 대화') & (df_copy['confidence'] < confidence_threshold)
    df_copy.loc[mask, 'prediction'] = '일반 대화'

    print(f"재분류된 샘플 수: {mask.sum()}개")
    return df_copy

# 사용
df_improved = reclassify_low_confidence(df, confidence_threshold=0.75)

In [ ]:
# 임계값 부여 방식 분류
# 현재: 가장 높은 확률의 클래스를 선택
predicted_class = np.argmax(predictions)

# 개선: 임계값 조정
def predict_with_threshold(predictions, thresholds):
    """
    thresholds: {0: 0.6, 1: 0.5, 2: 0.5, 3: 0.5, 4: 0.3}
    갈취(1)는 높은 확률 필요, 일반(4)은 낮은 확률로도 허용
    """
    max_class = np.argmax(predictions)
    max_prob = predictions[max_class]

    # 일반 대화 우선 검토
    if predictions[4] > thresholds[4]:  # 일반이 30% 이상이면
        return 4

    # 갈취는 엄격하게
    if max_class == 1 and max_prob < thresholds[1]:  # 갈취가 50% 미만이면
        return 4  # 일반으로 분류

    return max_class

# 사용 예시
thresholds = {
    0: 0.6,  # 협박
    1: 0.7,  # 갈취 (엄격하게!)
    2: 0.6,  # 직장 내 괴롭힘
    3: 0.6,  # 기타 괴롭힘
    4: 0.2   # 일반 (관대하게!)
}